In [13]:
import json
from pyspark import SparkContext
from pyspark.sql import SparkSession

In [14]:
sc = SparkContext()
ss = SparkSession(sc)


In [32]:
business = ss.read.option("multiline", "true").json("data/yelp_academic_dataset_business_clean.json")
checkin = ss.read.option("multiline", "true").json("data/yelp_academic_dataset_checkin_clean.json")
review = ss.read.option("multiline", "true").json("data/yelp_academic_dataset_review_clean.json").cache()
tip = ss.read.option("multiline", "true").json("data/yelp_academic_dataset_tip_clean.json")
user = ss.read.option("multiline", "true").json("data/yelp_academic_dataset_user_clean.json")


In [36]:
#print(business.head())
#print(checkin.head())
#print(review.head())
#print(tip.head())
#print(user.head())

review.head()


AnalysisException: 
Since Spark 2.3, the queries from raw JSON/CSV files are disallowed when the
referenced columns only include the internal corrupt record column
(named _corrupt_record by default). For example:
spark.read.schema(schema).csv(file).filter($"_corrupt_record".isNotNull).count()
and spark.read.schema(schema).csv(file).select("_corrupt_record").show().
Instead, you can cache or save the parsed results and then send the same query.
For example, val df = spark.read.schema(schema).csv(file).cache() and then
df.filter($"_corrupt_record".isNotNull).count().
      